In [1]:
import numpy as np
import pandas as pd
import math
from timeit import default_timer as timer

start=timer()

def distance(i1, i2):
        dist=0
        for j in range(6,m):
                dist=dist+(a[j][i1]-a[j][i2])*(a[j][i1]-a[j][i2])
        return(math.sqrt(dist))

def rank(values,dim):
        sortedv=list(range(dim))
        sortedi=list(range(dim))
        inici=0
        while values[inici]<0: 
            inici=inici+1
        sortedv[0]=values[inici]
        sortedi[0]=inici
#Number of values already sorted (among those without group)
        i1=0
        for i in range(inici,dim):
            j=0
            if values[i]>=0:    
                while (j<i1) and (sortedv[j] <= values[i]):
                        j=j+1   
                #i to be inserted before j 
                if sortedv[j] > values[i]:
#                    sortedv[j+1]=sortedv[j]
#                    sortedi[j+1]=sortedi[j]                    
                    for c in range(i1-1,j-1,-1):
                            sortedv[c+1]=sortedv[c]
                            sortedi[c+1]=sortedi[c]
                    sortedv[j]=values[i]
                    sortedi[j]=i
                else:
                    sortedv[i1]=values[i]
                    sortedi[i1]=i
                i1=i1+1
#       for i in range(dim):
#             sortedi[i]=sortedi[i]+1
        return sortedv,sortedi

    
def rank2(values,dim):
    sortedv=list(range(dim))
    sortedi=list(range(dim))
    sortedv[0]=values[0]
    sortedi[0]=0
#Number of values already sorted
    i1=0
    for i in range(0,dim):
            j=0
            while (j<i1) and (sortedv[j] <= values[i]):
                j=j+1   
                #i to be inserted before j 
            if sortedv[j] > values[i]:
#                    sortedv[j+1]=sortedv[j]
#                    sortedi[j+1]=sortedi[j]                    
                for c in range(i1-1,j-1,-1):
                        sortedv[c+1]=sortedv[c]
                        sortedi[c+1]=sortedi[c]
                sortedv[j]=values[i]
                sortedi[j]=i
            else:
                sortedv[i1]=values[i]
                sortedi[i1]=i
            i1=i1+1
#    for i in range(dim):
#            sortedi[i]=sortedi[i]+1
    return sortedv,sortedi

#ID=csv_read['ID']
#Y1=csv_read['Y1']
#Y2=csv_read['Y2']
#Y3=csv_read['Y3']
#Y4=csv_read['Y4']
#Y5=csv_read['Y5']
#SY4=csv_read['SY4']
#SY5=csv_read['SY5']

k=int(input("Enter k: "))
#number of attributes m
m=8
inFile = open('synthetic.csv','r')
outFile= open('microaggregated'+str(k)+'.csv', 'w')

a=list(range(0))
avg=list(range(0))
for j in range(m):
    a.append(list(range(0)))
    avg.append(float(0))         
             
linia=inFile.readline()
linia=inFile.readline()
i=0
while linia != '':
    valors=linia.split(',')
    a[0].append(int(valors[0]))
    for j in range(1,m):
        a[j].append(float(valors[j]))
    i=i+1
    linia=inFile.readline()
n=i

#One more position to store the average record
for j in range(m):
    a[j].append(float(0))

assigned_group=list(range(n))
for i in range(n):
    assigned_group[i]=0
current_group=0
distvalue=list(range(n))

#nprime is the number of records among n that remain to be assigned to a group
nprime=n
while nprime >=3*k:
    #Compute avg of unassigned records and store it as record in row n
    for j in range(6,m):
        a[j][n]=float(0)
    nprimecheck=0    
    for j in range(6,m):
        for i in range(n):
            if assigned_group[i]==0:
                a[j][n]=a[j][n]+a[j][i]
        a[j][n]=a[j][n]/nprime
##    print('nprime '+str(nprime))
    #Compute x_r
    r=0
    while assigned_group[r]!=0:
        r=r+1
    maximr=distance(r,n)
    for i in range(n):
        if assigned_group[i]==0:
            if distance(i,n) > maximr:
                        r=i
                        maximr=distance(i,n)
    #Compute x_s
    s=0
    while assigned_group[r]!=0:
        s=s+1
    maxims=distance(s,r)
    for i in range(n):
        if assigned_group[i]==0:
            if distance(i,r) > maxims:
                    s=i
                    maxims=distance(i,r)    
##    print('r:'+str(r)+' s:'+str(s))
    #Find k records closest to x_r and form a group with them
    for i in range(n):
        if assigned_group[i]!=0:
            distvalue[i]=-1
        else:
            distvalue[i]=distance(i,r)
    sortedv,sortedi=rank(distvalue,n)
    current_group=current_group+1
    for i in range(k):
            assigned_group[sortedi[i]]=current_group
##            print('x_r'+str(',')+str(sortedi[i])+str(',')+str(assigned_group[sortedi[i]]))
    #Find k records closest to x_s and form a group with them
    for i in range(n):
        if assigned_group[i]!=0:
            distvalue[i]=-1
        else:
            distvalue[i]=distance(i,s)
    sortedv,sortedi=rank(distvalue,n)
    current_group=current_group+1
    for i in range(k):
            assigned_group[sortedi[i]]=current_group
 ##           print('x_s'+str(',')+str(sortedi[i])+str(',')+str(assigned_group[sortedi[i]]))
    #Count records unassigned to groups
    nprime=0
    for i in range(0,n):
        if assigned_group[i]==0:
                nprime=nprime+1
##    print('nprime'+str(nprime))
    
if (nprime >= 2*k and nprime <= 3*k-1):
    #Compute avg of unassigned records and store it as record in row n
    for j in range(6,m):
        a[j][n]=float(0)
    for j in range(6,m):
        for i in range(n):
            if assigned_group[i]==0:
                a[j][n]=a[j][n]+a[j][i]
        a[j][n]=a[j][n]/nprime
#Compute x_r
    r=0
    while assigned_group[r]!=0:
        r=r+1
    maximr=distance(r,n)
    for i in range(n):
        if assigned_group[i]==0:
            if distance(i,n) > maximr:
                r=i
                maximr=distance(i,n)
#Find k records closest to x_r and form a group with them
    for i in range(n):
        if assigned_group[i]!=0:
            distvalue[i]=-1
        else:
            distvalue[i]=distance(i,r)
    sortedv,sortedi=rank(distvalue,n)
    current_group=current_group+1
    for i in range(k):
        assigned_group[sortedi[i]]=current_group
##        print(assigned_group[sortedi[i]])
    #Form a group with the remaining records
    current_group=current_group+1
    for i in range(n):
        if assigned_group[i]==0:
            assigned_group[i]=current_group    
else:
# nprime <2k
    current_group=current_group+1
    for i in range(n):
        if assigned_group[i]==0:
            assigned_group[i]=current_group
##            print(assigned_group[i])
    
# Write microaggregated file.    
linia='ID,Y1,Y2,Y3,Y4,Y5,SY4,SY5,group'+'\n'
outFile.write(linia)
for i in range(n):
    linia=''
    for j in range(m):
        linia=linia+str(a[j][i])+',' 
    linia=linia+str(assigned_group[i])+'\n'
    outFile.write(linia)

#print('current_group')
#print(current_group)
    
#create permuted data set: within-rank interval permutation for Y1, Y2, Y3, within microaggregation class for Y4-Y5.
#b=list(range(0))
#for j in range(m):
#    b.append(list(range(0)))

#Permutation for Y1, Y2, Y3. Look at paper to understand indices. 

##changed to debug
#n=10

for runs in range(5):
    outFile2= open('permuted'+str(k)+'_'+str(runs)+'.csv', 'w')
    b=np.zeros((m,n))
    #values and ranks before permutation
    attr=list(range(n))
    rankattr=list(range(n))
    #will contain permuted values and permuted ranks
    attrp=list(range(n))
    rankattrp=list(range(n))

#To obtain implementation of Algorithm 2 replace following instruction by "for j in range(1,6)"
    for j in range(1,4):
    ##for j in range(1,2):
    #sort attribute values
        for i in range(n):
            attr[i]=a[j][i]
    ##debug
    #    print(attr[i])
        sortedv,sortedi=rank2(attr,n)
    #    print("Sort results")
    #    print(sortedv)
    #    print(sortedi)
    #compute the rank of each attribute value
        for i in range(n):
            i1=0
            while (i1<n) and (sortedv[i1]!= attr[i]):
                i1=i1+1
    ##            print(str(sortedv[i1])+","+str(i))
            if (sortedv[i1]==attr[i]):
                rankattr[i]=i1
    #    print(attr)
    #    print(rankattr)
    #randomly permute within each group of k ranks for each attribute
        for i in range(0,k*math.floor(n/k-2)+1,k):
    #        print("i:")
    #        print(i)
    #        print("limit:")
    #        print(k*math.floor(n/k-2))
            intervranks=np.random.permutation(k)
            for i1 in range(k):
                i2=0
                while (rankattr[i2]!=i+i1):
                    i2=i2+1
                rankattrp[i2]=i+intervranks[i1]
    #            print(rankattr[i2],rankattrp[i2])
    #permute last group for each attribute (may contain more than k ranks)
        intervranks=np.random.permutation(n-k*math.floor(n/k-1))
        for i1 in range(n-k*math.floor(n/k-1)):
            i2=0
    ##        print(k*math.floor(n/k-1)+1+i1)
            while (rankattr[i2]!=k*math.floor(n/k-1)+i1):
                    i2=i2+1
            rankattrp[i2]=k*math.floor(n/k-1)+intervranks[i1]
    #assign the values that correspond to the permuted ranks
        for i in range(n):
            i1=0
            while (rankattr[i1]!=rankattrp[i]):
                i1=i1+1
            attrp[i]=attr[i1]
            b[j][i]=attr[i1]
    #    print(attrp)
    #    print(b[j])
    #    print(rankattrp)
    
    #Permutation for Y4-Y5 within microaggregation groups.
    #To obtain implementation of Algorithm 2 comment the following instructions up to the one marked "End_comment"
    maxgroups=current_group
    for current_group in range(1,maxgroups+1):
        for i1 in range(n):
            rankattr[i1]=0
        i1=0
    #identify the indices of the values in the current group (i1 will be the number of values in the group)
        for i in range(n):
            if assigned_group[i]==current_group:
                rankattr[i1]=i
                i1=i1+1
    #generate random permutation for the group
        intervranks=np.random.permutation(i1)
    #    print('intervranks')
    #    print(intervranks)
        for i in range(i1):
    #        print('i'+str(i))
    #        print(intervranks[i])
    #        print(rankattr[i])
    #        print(rankattr[intervranks[i]])
            b[4][rankattr[i]]=a[4][rankattr[intervranks[i]]]
            b[5][rankattr[i]]=a[5][rankattr[intervranks[i]]]
            b[6][rankattr[i]]=current_group
    #print(a[4])
    #print(a[5])
    #print(b[4])
    #print(b[5])
#End_comment
    
    # Write permuted file.    
    linia='Y1,Y2,Y3,Y4,Y5,group'+'\n'
    outFile2.write(linia)
    for i in range(n):
        linia=''
        linia=linia+str(b[1][i])+','
        linia=linia+str(b[2][i])+','
        linia=linia+str(b[3][i])+','
        linia=linia+str(int(b[4][i]))+','
        linia=linia+str(int(b[5][i]))+','
        linia=linia+str(int(b[6][i]))+'\n'
        outFile2.write(linia)
print("Finished")
end = timer()
print(end-start)

Enter k: 3
Finished
12.702474624995375
